In [106]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import pandas as pd
import numpy as np
from ambiente import usedAmbiente

In [108]:
coordPoints = pd.read_csv(usedAmbiente.coord, sep='\t', header=None, names=['id', 'coord'])

In [109]:
trace = pd.read_csv(usedAmbiente.trace, sep='\t', header=None, names=['timestamp', 'transmissor', 'receptor', 'snr'])


In [110]:
trace = trace.sort_values(['receptor','transmissor','timestamp']) #ordena na ordem dos parãmetros
aps = ['00:00:00:00:00:01','00:00:00:00:00:02','00:00:00:00:00:03','00:00:00:00:00:04','00:00:00:00:00:05','00:00:00:00:00:06','00:00:00:00:00:07']
trace = trace[~trace.receptor.isin(aps)]

In [111]:
coordPoints = coordPoints.iloc[7:]

In [112]:
trace['time'] = np.floor(trace.timestamp) #corta o tempo em inteiro


In [113]:
groupedByTime = trace.groupby(['receptor','transmissor','time'], as_index= False).agg({'snr':'mean'}) #calcula a media do snr para cada par receptor-transmissor no tempo X

In [114]:
groupedByReceptor = groupedByTime.pivot(columns='transmissor', index=['receptor','time'],values='snr') #indexa o snr de cada transmissor ao par receptor-tempo

In [115]:
groupedByReceptor= groupedByReceptor-98 # calcula o dbm

In [116]:
groupedByReceptor

transmissor               00:00:00:00:00:01  00:00:00:00:00:02  \
receptor          time                                           
00:00:00:00:00:08 0.0            -71.893186         -59.786920   
                  1.0            -69.132070         -55.963778   
                  2.0            -71.004220         -59.399933   
                  3.0            -69.182789         -62.400760   
                  4.0            -68.893667         -60.929950   
...                                     ...                ...   
00:00:00:00:00:58 1020.0         -65.123600         -78.521633   
                  1021.0         -62.374270         -80.133071   
                  1022.0         -61.335140         -80.674550   
                  1023.0         -64.849020         -79.036400   
                  1024.0                NaN                NaN   

transmissor               00:00:00:00:00:03  00:00:00:00:00:04  \
receptor          time                                           
00:00:00:00:00:08 0.0            -68.146180         -79.928400   
                  1.0            -65.276150         -83.257540   
                  2.0            -66.649620         -83.565167   
                  3.0            -68.245233         -85.281800   
                  4.0            -65.468310         -83.046550   
...                                     ...                ...   
00:00:00:00:00:58 1020.0         -71.706900         -28.189070   
                  1021.0         -73.689310         -28.842670   
                  1022.0         -74.608000         -34.331640   
                  1023.0         -75.166140         -29.815922   
                  1024.0                NaN                NaN   

transmissor               00:00:00:00:00:05  00:00:00:00:00:06  \
receptor          time                                           
00:00:00:00:00:08 0.0            -50.188320         -64.117260   
                  1.0            -46.401290         -68.384530   
                  2.0            -47.632211         -66.301170   
                  3.0            -46.412690         -66.586330   
                  4.0            -48.156240         -64.172511   
...                                     ...                ...   
00:00:00:00:00:58 1020.0         -82.438288         -77.090143   
                  1021.0         -83.131940         -78.075040   
                  1022.0         -80.943033         -80.356288   
                  1023.0         -82.918600         -77.784014   
                  1024.0                NaN                NaN   

transmissor               00:00:00:00:00:07  
receptor          time                       
00:00:00:00:00:08 0.0            -64.171633  
                  1.0            -64.292930  
                  2.0            -68.022810  
                  3.0            -65.435930  
                  4.0            -63.317589  
...                                     ...  
00:00:00:00:00:58 1020.0         -74.863856  
                  1021.0         -75.249700  
                  1022.0         -73.241890  
                  1023.0         -75.278300  
                  1024.0         -77.938200  

[83024 rows x 7 columns]

In [117]:
#groupedByReceptor.isna().groupby(level=0).sum().to_csv('nan.tsv', '\t') #visualiza quantos NAN tem AP-RP
#groupedByReceptor[groupedByReceptor['id'].isna()].receptor.unique()

In [118]:
groupedByReceptor = groupedByReceptor.dropna() #apaga todos os NAN
groupedByReceptor = groupedByReceptor.apply(lambda line: ','.join(line.astype(str)),axis=1) #junta todos os dbm para formar o vetor fingerprint
groupedByReceptor = groupedByReceptor.rename('fingerprint') 

In [119]:
coordPoints.id +=8 #acerta o index para que a contagem de transmissores e receptores seja continua
coordPoints.id = coordPoints.id.map(lambda id : '00:00:00:00:00:{:02X}'.format(id).lower()) #formata e converte o index para HEX

In [120]:
groupedByReceptor= groupedByReceptor.reset_index() #receptor e time voltam a ser colunas

In [121]:
fingerprint = groupedByReceptor.merge(coordPoints,how='left',  left_on='receptor', right_on='id') #juntas as duas tabelas
fingerprint

,receptor,time,fingerprint,id,coord
0,00:00:00:00:00:08,0.0,"-71.89318571428572,-59.786919999999995,-68.146...",00:00:00:00:00:08,1.11111:1.11111:1
1,00:00:00:00:00:08,1.0,"-69.13207,-55.96377777777778,-65.27615,-83.257...",00:00:00:00:00:08,1.11111:1.11111:1
2,00:00:00:00:00:08,2.0,"-71.00422,-59.39993333333334,-66.64962,-83.565...",00:00:00:00:00:08,1.11111:1.11111:1
3,00:00:00:00:00:08,3.0,"-69.1827888888889,-62.400760000000005,-68.2452...",00:00:00:00:00:08,1.11111:1.11111:1
4,00:00:00:00:00:08,4.0,"-68.89366666666666,-60.929950000000005,-65.468...",00:00:00:00:00:08,1.11111:1.11111:1
...,...,...,...,...,...
81475,00:00:00:00:00:58,1019.0,"-63.54047777777778,-76.8228125,-74.6525,-29.63...",00:00:00:00:00:58,18.8889:18.8889:1
81476,00:00:00:00:00:58,1020.0,"-65.1236,-78.52163333333334,-71.7069,-28.18907...",00:00:00:00:00:58,18.8889:18.8889:1
81477,00:00:00:00:00:58,1021.0,"-62.37427,-80.13307142857143,-73.68931,-28.842...",00:00:00:00:00:58,18.8889:18.8889:1
81478,00:00:00:00:00:58,1022.0,"-61.33514,-80.67455,-74.608,-34.33163999999999...",00:00:00:00:00:58,18.8889:18.8889:1


In [122]:
fingerprint = fingerprint[['coord', 'fingerprint']] #remove os campos extras

In [123]:
fingerprint.to_csv(usedAmbiente.exitFile, '\t', index=False, header=False)

C:\Users\mayla\AppData\Local\Temp\ipykernel_23276\1432663560.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_csv except for the argument 'path_or_buf' will be keyword-only.
  fingerprint.to_csv(usedAmbiente.exitFile, '\t', index=False, header=False)
